In [3]:
#TAREA 1 CLAUDIA DONOSO Y JAVIERA KAID

In [1]:
from finrisk import QC_Financial_3 as Qcf
from dataclasses import dataclass
from enum import Enum
from IPython.display import Image
from IPython.core.display import HTML
import pandas as pd
from datetime import date

# Modificado por AD
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

import numpy as np
import sys
from scipy.optimize import root_scalar
from functools import partial
from enum import Enum

In [2]:
class BusCal(Enum):
    NY = 1
    SCL = 2

In [3]:
def get_cal(code: BusCal) -> Qcf.BusinessCalendar:

    if code == BusCal.NY:
        cal = Qcf.BusinessCalendar(Qcf.QCDate(1, 1, 2020), 20)
        for agno in range(2020, 2071):
            f = Qcf.QCDate(12, 10, agno)
            if f.week_day() == Qcf.WeekDay.SAT:
                cal.add_holiday(Qcf.QCDate(14, 10, agno))
            elif f.week_day() == Qcf.WeekDay.SUN:
                cal.add_holiday(Qcf.QCDate(13, 10, agno))
            elif f.week_day() == Qcf.WeekDay.MON:
                cal.add_holiday(Qcf.QCDate(12, 10, agno))
            elif f.week_day() == Qcf.WeekDay.TUE:
                cal.add_holiday(Qcf.QCDate(11, 10, agno))
            elif f.week_day() == Qcf.WeekDay.WED:
                cal.add_holiday(Qcf.QCDate(10, 10, agno))
            elif f.week_day() == Qcf.WeekDay.THU:
                cal.add_holiday(Qcf.QCDate(9, 10, agno))
            else:
                cal.add_holiday(Qcf.QCDate(8, 10, agno))
        cal.add_holiday(Qcf.QCDate(15, 2, 2021))
        
    return cal

In [4]:
data = pd.read_excel('../data/SOFR-10012019-10292020.xls',"nice format")

# Los campos de fecha se importan como datetime (con horas, minutos y segundos).
# Se mantiene sólo la fecha.
data['fecha_inicial'] = data['DATE']
data['tasa'] = data['RATE\n(PERCENT)'] * 0.01  # Transformamos las tasas de intereses a decimal.
data = data.drop(data.columns[[1]], axis='columns')
fecha = data.iloc[0:269,0]
addfecha = "2020-10-29"
fecha = np.append(addfecha, fecha)
data["fecha_final"]= pd.Series(fecha)
data= data.iloc[::-1]
data['fecha_inicial'] = pd.to_datetime(data['fecha_inicial'], errors='coerce')
data['fecha_final'] = pd.to_datetime(data['fecha_final'], errors='coerce')
data['fecha_inicial'] = data['fecha_inicial'].dt.date
data['fecha_final'] = data['fecha_final'].dt.date

# Se visualiza el resultado
#data.style.format({'RATE(PERCENT)': '{0:.2%}'})

In [5]:
start_date=Qcf.QCDate(2,12,2019)
accrual_date=Qcf.QCDate(2,1,2020)
fixing =  Qcf.time_series()
finales = Qcf.time_series()               
fecha_final = accrual_date
dia = start_date.day_diff(fecha_final)
fechas = []

for i in range(270):


                        fecha1 = Qcf.QCDate(data.iloc[i]['fecha_inicial'].day,
                                            data.iloc[i]['fecha_inicial'].month,
                                            data.iloc[i]['fecha_inicial'].year)
                        fechaF = Qcf.QCDate(data.iloc[i]['fecha_final'].day,
                                            data.iloc[i]['fecha_final'].month,
                                            data.iloc[i]['fecha_final'].year)
                        fechas = i

                        if fecha1 == start_date:

                             for j in range(fechas,fechas+dia):

                                fecha2 = Qcf.QCDate(data.iloc[j]['fecha_inicial'].day,
                                            data.iloc[j]['fecha_inicial'].month,
                                            data.iloc[j]['fecha_inicial'].year)

                                fechaF2 = Qcf.QCDate(data.iloc[j]['fecha_final'].day,
                                            data.iloc[j]['fecha_final'].month,
                                            data.iloc[j]['fecha_final'].year)




                                fixing[fecha2] = data.iloc[j]['tasa']
                                finales[fechaF2] = data.iloc[j]['tasa']
                                

In [6]:
class RAYEN:
    
    # AD: todo esto estaba incluido con el decorador @dataclass
    
    def __init__ (self, start_date, end_date, settlement_date, notional, currency, amortization,amort_is_cashflow,interest_rate, on_index, spread, gearing):
        
        self.start_date = start_date
        self.end_date = end_date
        self.settlement_date = settlement_date
        self.notional = notional
        self.currency = currency
        self.amortization = amortization
        self.amort_is_cashflow = amort_is_cashflow
        self.interest_rate = interest_rate
        self.on_index = on_index
        self.spread = spread
        self.gearing = gearing

    def get_accrued_rate(self, accrual_date: Qcf.QCDate, fixing: Qcf.time_series, finales: Qcf.time_series) -> float:

        self.fecha_final= accrual_date

        self.fl = Qcf.time_series_dates(finales)                     
        self.dl = Qcf.time_series_dates(fixing)                                     
        self.vl = Qcf.time_series_values(fixing)



        datos= pd.DataFrame({'FECHA INICIAL':self.dl,'FECHA FINAL':self.fl,'TASAS':self.vl})

        def get_wf(fecha_inicial: self.start_date, fecha_final: self.fecha_final, valor_tasa: float) -> float:
                            self.qc_tasa = Qcf.QCInterestRate(valor_tasa, Qcf.QCAct360(), Qcf.QCLinearWf())    
                            return self.qc_tasa.wf(fecha_inicial, fecha_final)

        datos['wf'] = datos.apply(
        lambda row: get_wf(row['FECHA INICIAL'], row['FECHA FINAL'],row['TASAS']),
        axis=1
        )

        datos.style.format({'TASAS': '{0:.2%}', 'wf': '{0:.6%}'})
        #Producto de los factores de capitalización

        self.P = datos['wf'].product()
        self.dias = self.start_date.day_diff(self.fecha_final)
                        # Tasa equivalente
                        # prod = 1 + tasa_eq * dias / 360
        self.tasa_eq = (self.P - 1) * 360.0 / self.dias

        return self.tasa_eq

 
    
    def get_accrued_interest(self, accrual_date: Qcf.QCDate, fixings: Qcf.time_series) -> float:
        
        self.interes_flot_ois = self.notional * self.tasa_eq * self.dias / 360     
        return self.interes_flot_ois       
        
    
    def amount(self, fixings: Qcf.time_series) -> float:
        self.qc_tasa_eq = Qcf.QCInterestRate(self.tasa_eq, Qcf.QCAct360(), Qcf.QCLinearWf())
        self.flujo = self.amortization + self.notional * (self.qc_tasa_eq.wf(self.start_date, self.fecha_final) - 1)
        return self.flujo
    
    def test(self):
        return 'test'
    
    def test1(self, x: float):
        if x == 0:
            raise ValueError('x debe ser != 0')
        return 1 / x
    
    def test2(self, x: float):
        return self.test1(x) + 1
    
    def test3(self, x: float):
        return x + self.notional

In [7]:
codigo = 'SOFR'
tasa_on = Qcf.QCInterestRate(.0, Qcf.QCAct360(), Qcf.QCLinearWf())
fixing_lag = Qcf.Tenor('0d')
tenor = Qcf.Tenor('1d')
fixing_calendar = get_cal(BusCal.NY)     
settlement_calendar = fixing_calendar  
sofr = Qcf.InterestRateIndex(
    codigo,
    tasa_on,
    fixing_lag,
    tenor,
    fixing_calendar,
    settlement_calendar,
    Qcf.QCUSD()
)

In [8]:
ois = RAYEN(
    Qcf.QCDate(2,12,2019),
    Qcf.QCDate(2, 1, 2020),
    Qcf.QCDate(13, 1, 2021),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

In [9]:
ois.get_accrued_rate(Qcf.QCDate(2,1,2020),fixing,finales)

0.022453029017159128

In [10]:
ois.get_accrued_interest(Qcf.QCDate(2,1,2020),fixing)

19334.552764775915

In [11]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')
#print(df_curva)

In [12]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

In [13]:
 frmt = {
    'tasa': '{:.6%}',
    'df': '{:.6%}',
    'valor_tasa': '{:.4%}',
    'spread': '{:.4%}',
    'nominal': '{:,.2f}',
    'interes': '{:,.2f}',
    'amortizacion': '{:,.2f}',
    'flujo': '{:,.2f}',
}

In [14]:
def set_expected_rate(
        val_date: Qcf.QCDate,
        ois: RAYEN,
        zcc: Qcf.ZeroCouponCurve,
        fixing: Qcf.time_series) -> None:
         

    
        tasas=[]                            
        vl = Qcf.time_series_values(fixing)
        for v in vl:
            tasas.append(v)
        
        get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)
     
        op = get_ois_sofr(
        rp=Qcf.RecPay.RECEIVE,
        notional=ois.notional,
        start_date=ois.start_date,
        tenor=Qcf.Tenor('2Y6M'),
        fixed_rate_value=.01,
        spread=ois.spread,
        gearing=ois.gearing)
    
        fwd = Qcf.ForwardRates()
        for t in tasas:
            tabla = fwd.set_rates_icp_clp_leg(val_date, t, op[1], zcc)
            return (aux.show_leg(op[1], 'IcpClpCashflow', '').style.format(frmt), tabla)
    

                        

In [15]:

(Tasas_esperadas, Tabla) = set_expected_rate(Qcf.QCDate(4, 10, 2020), ois, zcc, fixing)



In [16]:
Tasas_esperadas

,fecha_inicial,fecha_final,fecha_pago,nominal,amortizacion,amort_es_flujo,flujo,moneda,icp_inicial,icp_final,valor_tasa,interes,spread,gearing,tipo_tasa
0,2019-12-02,2020-06-02,2020-06-02,"-10,000,000.00",0.00,True,0.00,CLP,1.000000,1.000000,0.0000%,-0.00,0.0000%,1.000000,LinAct360
1,2020-06-02,2021-06-02,2021-06-02,"-10,000,000.00",0.00,True,"9,836,921.93",CLP,1.000000,0.016308,-97.0200%,"9,836,750.00",0.0000%,1.000000,LinAct360
2,2021-06-02,2022-06-02,2022-06-02,"-10,000,000.00","-10,000,000.00",True,"-10,005,466.13",CLP,0.016308,0.016317,0.0500%,"-5,069.44",0.0000%,1.000000,LinAct360


In [17]:
def present_value(val_date,ois,zcc) -> float:

    get_ois_sofr = partial(aux.get_ois_using_template, aux.type_ois_template, aux.TypeOis.SOFR)
    op = get_ois_sofr(
        rp=Qcf.RecPay.RECEIVE,
        notional=ois.notional,
        start_date=ois.start_date,
        tenor=Qcf.Tenor('2Y6M'),
        fixed_rate_value=.01,
        spread=ois.spread,
        gearing=ois.gearing)
    
    valor_presente = Qcf.PresentValue()


    vp_flot = valor_presente.pv(val_date, op[1], zcc)
    vp_fija = valor_presente.pv(val_date, op[0], zcc)
    valor_total=vp_fija+vp_flot
    
    return (valor_total, vp_flot)

In [18]:
(valor_total, pata_flotante) = present_value(Qcf.QCDate(14, 10, 2020), ois, zcc)

In [19]:
print(f'El valor presente de la pata flotante es: USD {pata_flotante:,.2f}')
print(f'Por lo tanto, el valor total de la operación es:\nValor total: USD {valor_total:,.2f}')

El valor presente de la pata flotante es: USD -9,989,855.30
Por lo tanto, el valor total de la operación es:
Valor total: USD 202,628.17


## Tests

In [20]:
import sys
sys.path.insert(1, '../modules')
import auxiliary as aux

In [21]:
amount_tol = 10000
rate_tol = .001

In [22]:
exitos = 0

In [23]:
def suma_exito(resultado, check, tipo):
    global exitos
    if resultado is None:
        return
    else:
        print('Suma medio punto por obtener resultado.')
        exitos += .5
        if tipo == 'monto':
            if abs(resultado - check) < amount_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1
        else:
            if abs(resultado - check) < rate_tol:
                print('Suma 1 punto por obtener resultado dentro de la tolerancia.')
                exitos += 1

Construcción de una instancia de `OisCashflow`.

In [25]:
ois = RAYEN(
    Qcf.QCDate(1, 10, 2019),
    Qcf.QCDate(1, 10, 2020),
    Qcf.QCDate(1, 10, 2020),
    10000000,
    Qcf.QCUSD(),
    1000000,
    True,
    Qcf.QCInterestRate(0.0, Qcf.QCAct360(), Qcf.QCLinearWf()),
    sofr,
    0,
    1
)

En la siguiente variable `exitos` se registra cuantos tests se superan con éxito.

### Objeto `fixings`

In [26]:
df_fixings = pd.read_excel('../data/SOFR-10012019-10292020.xls', sheet_name='nice format')
df_fixings.columns = ['fecha', 'nombre', 'valor']
df_fixings['valor'] /= 100

In [27]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


In [28]:
fixings = Qcf.time_series()
for row in df_fixings.itertuples():
    fixings[Qcf.build_qcdate_from_string(row.fecha)] = row.valor

In [29]:
fixings[Qcf.QCDate(13, 1, 2020)]

0.0154

### `OisCashflow.accrued_rate`

In [31]:
accrued_rate = ois.get_accrued_rate(Qcf.QCDate(15, 6, 2020), fixings, fixings)
print(f'accrued rate: {accrued_rate:.8%}')

accrued rate: 0.00000000%


Se verificará el cálculo usando `df_fixings`.

In [32]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor
0,2020-10-28,SOFR,0.0800%
1,2020-10-27,SOFR,0.0900%
2,2020-10-26,SOFR,0.0900%
3,2020-10-23,SOFR,0.0800%
4,2020-10-22,SOFR,0.0700%


Se agrega la columna `next_date`, servirá para calcular el factor de capitalización de cada tasa.

In [33]:
def next_date(fecha:str, calendario: Qcf.BusinessCalendar) -> str:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    next_fecha = calendario.shift(qfecha, 1)
    return next_fecha.description(False)

In [34]:
df_fixings['next_fecha'] = df_fixings.apply(
    lambda row: next_date(row['fecha'], settlement_calendar),
    axis=1
)

In [35]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha
0,2020-10-28,SOFR,0.0800%,2020-10-29
1,2020-10-27,SOFR,0.0900%,2020-10-28
2,2020-10-26,SOFR,0.0900%,2020-10-27
3,2020-10-23,SOFR,0.0800%,2020-10-26
4,2020-10-22,SOFR,0.0700%,2020-10-23


Se calcula ahora el factor de capitalización.

In [36]:
def factor_cap(fecha: str, next_fecha: str, valor: float) -> float:
    qfecha = Qcf.build_qcdate_from_string(fecha)
    qnext_fecha = Qcf.build_qcdate_from_string(next_fecha)
    int_rate = Qcf.QCInterestRate(valor, Qcf.QCAct360(), Qcf.QCLinearWf())
    return int_rate.wf(qfecha, qnext_fecha)

In [37]:
df_fixings['factor_capitalizacion'] = df_fixings.apply(
    lambda row: factor_cap(row['fecha'], row['next_fecha'], row['valor']),
    axis=1
)

In [38]:
df_fixings.head().style.format({'valor': '{:.4%}'})

,fecha,nombre,valor,next_fecha,factor_capitalizacion
0,2020-10-28,SOFR,0.0800%,2020-10-29,1.000002
1,2020-10-27,SOFR,0.0900%,2020-10-28,1.000002
2,2020-10-26,SOFR,0.0900%,2020-10-27,1.000002
3,2020-10-23,SOFR,0.0800%,2020-10-26,1.000007
4,2020-10-22,SOFR,0.0700%,2020-10-23,1.000002


Veamos donde están en `df_fixings` los valores para `start_date` de `ois` y la fecha 15-6-2020 y calculamos el producto de los factores de capitalización desde esa fecha hasta la última fecha de `df_flujos` (que es el primer registro). Luego, el cociente de los dos factores nos dará el factor entre ambas fechas. Finalmente, con ese factor, se calculará `accrued_rate`.

In [39]:
start_date = ois.start_date.description(False)
df_fixings[df_fixings.fecha == start_date]

,fecha,nombre,valor,next_fecha,factor_capitalizacion
269,2019-10-01,SOFR,0.0188,2019-10-02,1.000052


In [40]:
factor_largo = df_fixings.iloc[:270]['factor_capitalizacion'].prod()
print(factor_largo)
qstart_date = Qcf.build_qcdate_from_string('2019-10-01')

1.0075715527341267


In [41]:
df_fixings[df_fixings.fecha == '2020-06-15']

,fecha,nombre,valor,next_fecha,factor_capitalizacion
94,2020-06-15,SOFR,0.0009,2020-06-16,1.000002


In [42]:
factor_corto = df_fixings.iloc[:95]['factor_capitalizacion'].prod()
qaccrued_date = Qcf.build_qcdate_from_string('2020-06-15')

In [43]:
dias = qstart_date.day_diff(qaccrued_date)
check_accrued_rate = (factor_largo / factor_corto - 1.0) * 360.0 / dias
print(f'La diferencia es: {accrued_rate - check_accrued_rate:.8%}')

La diferencia es: -1.00991314%


In [44]:
suma_exito(accrued_rate, check_accrued_rate, 'tasa')

Suma medio punto por obtener resultado.


### `OisCashflow.accrued_interest`

In [45]:
accrued_interest = ois.get_accrued_interest(Qcf.QCDate(15, 6, 2020), fixings)
print(f'accrued interest: {accrued_interest:,.2f}')

accrued interest: 0.00


In [46]:
check_accrued_interest = ois.notional * accrued_rate * dias / 360.0
print(f'La diferencia es: {accrued_interest - check_accrued_interest:,.6f}')

La diferencia es: 0.000000


In [47]:
suma_exito(accrued_interest, check_accrued_interest, 'monto')

Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `OisCashflow.amount`

In [48]:
amount = ois.amount(fixings)
print(f'amount: {amount:,.2f}')

amount: 1,000,000.00


In [49]:
end_date = ois.end_date.description(False)
print(f'end date: {end_date}')
df_fixings[df_fixings.fecha == end_date]

end date: 2020-10-01


,fecha,nombre,valor,next_fecha,factor_capitalizacion
18,2020-10-01,SOFR,0.0008,2020-10-02,1.000002


In [50]:
factor_corto_2 = df_fixings.iloc[:19]['factor_capitalizacion'].prod()
dias_2 = qstart_date.day_diff(ois.end_date)
accrued_rate_amount = (factor_largo / factor_corto_2- 1.0) * 360.0 / dias_2
check_amount = ois.notional * accrued_rate_amount * dias_2 / 360.0 + ois.amortization
print(f'La diferencia es: {amount - check_amount:.6f}')

La diferencia es: -75027.045434


In [51]:
suma_exito(amount, check_amount, 'monto')

Suma medio punto por obtener resultado.


### `set_expected_rate`

Comenzamos cargando los valores de la curva cero cupón que se utilizará para hacer pruebas de `set_expected_rate` y `present_value`.

In [52]:
df_curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

In [53]:
df_curva.head().style.format({'tasa': '{:.4%}', 'df': '{:.6%}'})

,plazo,tasa,df
0,1,0.0811%,99.999778%
1,7,0.0841%,99.998388%
2,14,0.0780%,99.997010%
3,21,0.0774%,99.995549%
4,33,0.0781%,99.992942%


Con la data se construye un objeto de tipo `Qcf.ZeroCouponCurve`.

In [54]:
zcc = aux.get_curve_from_dataframe(Qcf.QCAct365(),Qcf.QCCompoundWf(), df_curva)

Comienzan los tests.

In [55]:
print(f'Fecha inicial flujo: {ois.start_date.description(False)}')
print(f'Fecha final flujo: {ois.end_date.description(False)}')

Fecha inicial flujo: 2019-10-01
Fecha final flujo: 2020-10-01


In [57]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.06121277%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [58]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = val_date.day_diff(ois.start_date)
p2 = val_date.day_diff(ois.end_date)
df1 = zcc.get_discount_factor_at(p1)
df2 = zcc.get_discount_factor_at(p2)
check = (df1 / df2 - 1) * 360 / (p2 - p1)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.06924275%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [59]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
p1 = ois.start_date.day_diff(val_date)
p2 = val_date.day_diff(ois.end_date)
wf1 = 1 + ois.get_accrued_interest(val_date, fixings) / ois.notional
wf2 = 1 / zcc.get_discount_factor_at(p2)
check = (wf1 * wf2 - 1) * 360 / (p1 + p2)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.06395685%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [61]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(val_date, fixings, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.00000000%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


In [63]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
resultado = ois.interest_rate.get_value()
print(f'Tasa esperada es: {resultado:.8%}')
check = ois.get_accrued_rate(ois.end_date, fixings, fixings)
print(f'Check tasa esperada es: {check:.8%}')

suma_exito(resultado, check, 'tasa')

Tasa esperada es: 0.00000000%
Check tasa esperada es: 0.00000000%
Suma medio punto por obtener resultado.
Suma 1 punto por obtener resultado dentro de la tolerancia.


### `present_value`

In [64]:
# Check primer caso
val_date = Qcf.QCDate(1, 6, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: unsupported format string passed to tuple.__format__

In [65]:
# Check segundo caso borde
val_date = Qcf.QCDate(1, 10, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: unsupported format string passed to tuple.__format__

In [66]:
# Check segundo caso
val_date = Qcf.QCDate(1, 11, 2019)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: unsupported format string passed to tuple.__format__

In [67]:
# Check tercer caso borde
val_date = Qcf.QCDate(1, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: unsupported format string passed to tuple.__format__

In [68]:
# Check tercer caso
val_date = Qcf.QCDate(15, 10, 2020)
set_expected_rate(val_date, ois, zcc, fixings)
pv = present_value(val_date, ois, zcc)
print(f'Valor presente es: {pv:,.4f}')
amount = ois.amount()
print(f'Amount: {amount:,.4f}')
amount = ois.amount()
plazo = val_date.day_diff(ois.settlement_date)
if plazo <= 0:
    df = 0.0
else:
    df = zcc.get_discount_factor_at(plazo)
print(f'Df: {df:.6%}')
vp_check = df * amount
print(f'VP check es: {vp_check:,.4f}')

suma_exito(pv, vp_check, 'monto')

TypeError: unsupported format string passed to tuple.__format__

In [69]:
print(f'Éxitos totales: {exitos}')

Éxitos totales: 10.0
